In [2]:
#we will be looking at self-attention
#consider the following example 

import torch
torch.manual_seed(1337)
B,T,C = 4,8,2 #batch,time,channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [ ]:
#we want x[b, t] = mean_{i<=t} x[b,i]
xbow = torch.zero((B,T,C))
#bow is 'bag of words'